In [37]:
# %load_ext autoreload
# %autoreload 2
%matplotlib ipympl
import json
import time

import ipywidgets as widgets
import laser_pointer as lp
import matplotlib.pyplot as plt
import mpl_interactions.ipyplot as iplt
import numpy as np
from mpl_interactions import hyperslicer
from pycromanager import Acquisition, multi_d_acquisition_events
from raman_control import *
from tqdm.auto import tqdm

C:\Users\Raman\miniconda3\envs\control\lib\site-packages\matplotlib\_pylab_helpers.py:89: ResourceWarning: unclosed file <_io.BufferedReader name='z-stack-test/heck_1\\Full resolution\\heck_MagellanStack.tif'>
  gc.collect(1)


In [2]:
bridge, core, snap_image = setup_scope()

In [80]:
core.get_position()

4926.25

In [4]:
x = np.arange(0, 5)
y = np.arange(0, -5, -1)

xy = np.hstack([x[:, None], y[:, None]])
xy

array([[ 0,  0],
       [ 1, -1],
       [ 2, -2],
       [ 3, -3],
       [ 4, -4]])

In [98]:
x = core.get_x_position()
y = core.get_y_position()
z = core.get_position()
from sidecar import Sidecar

sc = Sidecar(title="Function Output")

In [99]:
def make_z_args_asym(cur_z, minus_rel, plus_rel, step):
    return {"z_start": cur_z - minus_rel, "z_end": cur_z + plus_rel, "z_step": step}


def make_z_args(cur_z, rel, step):
    return make_z_args_asym(cur_z, np.abs(rel), np.abs(rel), step)


# multi_d_acquisition_events(xy_positions=[[x, y]], z_start)

In [100]:
events = multi_d_acquisition_events(
    xy_positions=[[x, y]],
    **make_z_args(z, 5, 1),
    channel_group="UPLSAPO40X",
    channels=["BF", "GFP"]
)
# events = multi_d_acquisition_events(xy_positions=[[x,y]], channel_group='UPLSAPO40X', channels=['BF', 'GFP'])
events

[{'axes': {'position': 0, 'z': 0},
  'x': 51014,
  'y': 43547,
  'channel': {'group': 'UPLSAPO40X', 'config': 'BF'},
  'z': 4922.25},
 {'axes': {'position': 0, 'z': 1},
  'x': 51014,
  'y': 43547,
  'channel': {'group': 'UPLSAPO40X', 'config': 'BF'},
  'z': 4923.25},
 {'axes': {'position': 0, 'z': 2},
  'x': 51014,
  'y': 43547,
  'channel': {'group': 'UPLSAPO40X', 'config': 'BF'},
  'z': 4924.25},
 {'axes': {'position': 0, 'z': 3},
  'x': 51014,
  'y': 43547,
  'channel': {'group': 'UPLSAPO40X', 'config': 'BF'},
  'z': 4925.25},
 {'axes': {'position': 0, 'z': 4},
  'x': 51014,
  'y': 43547,
  'channel': {'group': 'UPLSAPO40X', 'config': 'BF'},
  'z': 4926.25},
 {'axes': {'position': 0, 'z': 5},
  'x': 51014,
  'y': 43547,
  'channel': {'group': 'UPLSAPO40X', 'config': 'BF'},
  'z': 4927.25},
 {'axes': {'position': 0, 'z': 6},
  'x': 51014,
  'y': 43547,
  'channel': {'group': 'UPLSAPO40X', 'config': 'BF'},
  'z': 4928.25},
 {'axes': {'position': 0, 'z': 7},
  'x': 51014,
  'y': 43547,

In [137]:
from collections import defaultdict

rm_data = defaultdict(dict)

In [144]:
rm_data = defaultdict(dict)
ev = events[0]
axes = ev["axes"]


def f():
    rm_data[axes["position"]][axes["z"]] = 0


# rm_data.get(**events[0]['axes'])
f()
rm_data

defaultdict(dict, {0: {0: 0}})

In [130]:
rm_data[events[0]["axes"]]

TypeError: unhashable type: 'dict'

In [108]:
events[0]["my-metadata"] = "heck"

In [145]:
rm_data = defaultdict(dict)
sc.clear_output()


def f(event):
    if event["axes"]["z"] == 5:
        with sc:
            rm_data[axes["position"]][axes["z"]] = np.zeros((10, 10))
            print(event)
    return event


with Acquisition(directory="z-stack-test", name="heck", post_hardware_hook_fn=f) as acq:
    dataset = acq.acquire(events)

defaultdict(dict,
            {0: {0: array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                     [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                     [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                     [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                     [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                     [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                     [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                     [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                     [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                     [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])}})

In [29]:
from pycromanager import Dataset

In [90]:
data = Dataset("z-stack-test/heck_15")

Dataset opened of 1


In [91]:
data = data.as_array()

In [94]:
plt.figure()
controls = hyperslicer(data, play_buttons=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\Raman\miniconda3\envs\control\lib\site-packages\matplotlib\cbook\__init__.py:92: ResourceWarning: unclosed file <_io.BufferedReader name='z-stack-test/heck_15\\Full resolution\\heck_MagellanStack.tif'>
  self._obj = obj


In [ ]:
out = widgets.Output()

In [111]:
galvo, shutter, open_shutter, close_shutter, close_daq = setup_laser_control()
auto, experiment, set_value, set_rm_exposure = setup_lightfield()
bridge, core, snap_image = setup_scope()


def shutdown():
    close_daq()
    auto.Dispose()

In [120]:
experiment.Preview()

In [113]:
shutter.open()

NameError: name 'shutter' is not defined

In [119]:
shutter.open()

In [121]:
shutter.close()

In [122]:
experiment.Stop()

In [124]:
make_grid(50)

(array([[-0.3      , -0.2877551, -0.2755102, ...,  0.2755102,  0.2877551,
          0.3      ],
        [-0.3      , -0.3      , -0.3      , ...,  0.3      ,  0.3      ,
          0.3      ]]),
 array([-0.3       , -0.2877551 , -0.2755102 , -0.26326531, -0.25102041,
        -0.23877551, -0.22653061, -0.21428571, -0.20204082, -0.18979592,
        -0.17755102, -0.16530612, -0.15306122, -0.14081633, -0.12857143,
        -0.11632653, -0.10408163, -0.09183673, -0.07959184, -0.06734694,
        -0.05510204, -0.04285714, -0.03061224, -0.01836735, -0.00612245,
         0.00612245,  0.01836735,  0.03061224,  0.04285714,  0.05510204,
         0.06734694,  0.07959184,  0.09183673,  0.10408163,  0.11632653,
         0.12857143,  0.14081633,  0.15306122,  0.16530612,  0.17755102,
         0.18979592,  0.20204082,  0.21428571,  0.22653061,  0.23877551,
         0.25102041,  0.26326531,  0.2755102 ,  0.2877551 ,  0.3       ]))

In [160]:
def collect_spectra(points, exposure=20):
    """
    Parameters
    ----------
    points : arraylike
        shape (2, N)
    exposure : float (Default: 20)
        exposure time in ms

    Returns
    -------
    spectra : array
        with shape (N, 1340)
    """
    points = np.asarray(points)
    galvo.stop()
    # xy_grid, volts = make_grid(N)
    SAMPLERATE = 100000
    SAMPLECLOCKSOURCE = "PFI0"
    galvo.timing.cfg_samp_clk_timing(
        SAMPLERATE,
        source=SAMPLECLOCKSOURCE,
        active_edge=nidaqmx.constants.Edge.FALLING,
        sample_mode=nidaqmx.constants.AcquisitionType.FINITE,
        samps_per_chan=points.shape[1],
    )
    galvo.write(points, auto_start=False)
    set_rm_exposure(exposure)

    galvo.stop()
    galvo.start()
    experiment.Stop()
    with open_shutter:
        dataset = experiment.Capture(points.shape[1])
    galvo.stop()
    return convert_capture(dataset)


def capture_rm_grid(N=75, exposure=20):
    """

    Parameters
    ----------
    N : int
        Length of the grid sides
    exposure : float
        exposure time in ms

    Returns
    -------
    spectra : array
        shape (N, N, 1340)
    volts : 1D array
        volts for passing to laser-pointer
    """
    xy_grid, volts = make_grid(N)
    return collect_spectra(xy_grid, exposure).squeeze().reshape(N, N, 1340), volts


def calibrate(N=75, exposure=20):
    """
    Parameters
    ----------
    N : int
        Dimension of the laser grid
    exposure : float
        Exposure time in ms

    Returns
    -------
    fit_calibration : function
        to be called when an adequate number of points have been selected
    lp_bf
    lp_rm
    """
    bf = snap_image()
    rm, volts = capture_rm_grid(N, exposure)
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    lp_bf = lp.pointer(bf, axes[0])
    lp_rm = lp.pointer(rm[:, :, 664], axes[1], X=volts, Y=volts, radius=0.005)

    def fit_calibration(plot=True):
        """
        Usage
        -----
        ``transform_pointer, bfx_to_rmx, bfy_to_rmy, inverse_x, inverse_y = fit_calibration()``
        """
        return fit_transform_pointers(lp_bf, lp_rm)

    return fit_calibration, lp_bf, lp_rm


def position_file_to_coords(fname):
    """
    parameters
    ----------
    fname : str
        The path the to the .pos from μmanager

    returns
    -------
    XY : 2D array
    Z : 1D array
    """
    with open(fname) as f:
        positions = json.load(f)
    xyz = positions["map"]["StagePositions"]["array"]
    pos = xyz[0]["DevicePositions"]["array"]
    X = []
    Y = []
    Z = []
    for i in range(len(xyz)):
        pos = xyz[i]["DevicePositions"]["array"]
        for i, p in enumerate(pos):
            device = p["Device"]
            if "FocusDrive" == device["scalar"]:
                # z
                z = p["Position_um"]["array"][0]
            elif "XYStage" == device["scalar"]:
                # xy
                x, y = p["Position_um"]["array"]
        X.append(x)
        Y.append(y)
        Z.append(z)
    xy_pos = np.zeros([len(X), 2])
    xy_pos[:, 0] = X
    xy_pos[:, 1] = Y
    return xy_pos, np.asarray(Z)


def collect_bf_images(xy, z=None):
    """
    parameters
    ---------
    xy : arraylike 2D
    z : arraylike or None
    """
    ims = []
    for i in range(len(xy)):
        core.set_xy_position(*xy[i])
        if z:
            core.set_position(z[i])
        ims.append(snap_image())
    return np.asarray(ims)

In [126]:
rm, volts = capture_rm_grid(25)

In [128]:
plt.figure()
plt.imshow(rm[:, :, 664])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [147]:
events

[{'axes': {'position': 0, 'z': 0},
  'x': 51014,
  'y': 43547,
  'channel': {'group': 'UPLSAPO40X', 'config': 'BF'},
  'z': 4922.25,
  'my-metadata': 'heck'},
 {'axes': {'position': 0, 'z': 1},
  'x': 51014,
  'y': 43547,
  'channel': {'group': 'UPLSAPO40X', 'config': 'BF'},
  'z': 4923.25},
 {'axes': {'position': 0, 'z': 2},
  'x': 51014,
  'y': 43547,
  'channel': {'group': 'UPLSAPO40X', 'config': 'BF'},
  'z': 4924.25},
 {'axes': {'position': 0, 'z': 3},
  'x': 51014,
  'y': 43547,
  'channel': {'group': 'UPLSAPO40X', 'config': 'BF'},
  'z': 4925.25},
 {'axes': {'position': 0, 'z': 4},
  'x': 51014,
  'y': 43547,
  'channel': {'group': 'UPLSAPO40X', 'config': 'BF'},
  'z': 4926.25},
 {'axes': {'position': 0, 'z': 5},
  'x': 51014,
  'y': 43547,
  'channel': {'group': 'UPLSAPO40X', 'config': 'BF'},
  'z': 4927.25},
 {'axes': {'position': 0, 'z': 6},
  'x': 51014,
  'y': 43547,
  'channel': {'group': 'UPLSAPO40X', 'config': 'BF'},
  'z': 4928.25},
 {'axes': {'position': 0, 'z': 7},
  

In [149]:
rm_data = defaultdict(dict)
sc.clear_output()


def f(event):
    if event["axes"]["z"] == 5 and event["channel"]["config"] == "BF":
        with sc:
            rm_data[axes["position"]][axes["z"]] = np.zeros((10, 10))
            print(event)
    return event


with Acquisition(directory="z-stack-test", name="heck", post_hardware_hook_fn=f) as acq:
    dataset = acq.acquire(events)

In [150]:
rm_data = defaultdict(dict)
sc.clear_output()


def f(event):
    if event["axes"]["z"] == 5 and event["channel"]["config"] == "BF":
        with sc:
            print("starting raman")
            rm_data[axes["position"]][axes["z"]] = capture_rm_grid(25)
            print("done raman")
            print(event)
    return event


with Acquisition(directory="z-stack-test", name="heck", post_hardware_hook_fn=f) as acq:
    dataset = acq.acquire(events)

In [154]:
arr = rm_data[0][0][0]
plt.figure()
plt.imshow(arr[:, :, 664])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [161]:
xy, z = position_file_to_coords("pos-oct28.pos")

In [162]:
np.array(z)

array([4926.25, 4928.25])

In [164]:
xyz = np.hstack([xy, z[:, None]])

In [165]:
xyz

array([[51014.  , 43547.  ,  4926.25],
       [50753.  , 43547.  ,  4928.25]])

In [169]:
events = multi_d_acquisition_events(
    xyz_positions=xyz,
    z_start=-5,
    z_end=5,
    z_step=1,
    channel_group="UPLSAPO40X",
    channels=["BF", "GFP"],
)

In [180]:
rm_data = defaultdict(dict)
sc.clear_output()

def f(event):
    if event["axes"]["z"] == 5 and event["channel"]["config"] == "BF":
        axes = event['axes']
        with sc:
            print("starting raman")
            print(axes['position'])
            rm_data[axes["position"]][axes["z"]] = capture_rm_grid(25)
            print("done raman")
            print(event)
    return event


with Acquisition(directory="z-stack-test", name="with-rm", post_hardware_hook_fn=f) as acq:
    dataset = acq.acquire(events)

In [183]:
fig, axs = plt.subplots(1,2, figsize=(10,5))
for p in [0,1]:
    arr = rm_data[p][5][0][:,:,664]
    axs[p].imshow(arr)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [179]:
rm_data[0].keys()

dict_keys([0])

In [184]:
shutdown()